In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import OneClassSVM
import joblib

# Example dataset
new_data = {
    'cirs': np.array(
        [
            [
                [
                    [1, 2],
                    [3, 4],
                    [5, 6]
                ],
                [
                    [9, 10],
                    [11, 12],
                    [13, 14]
                ],
                [
                    [17, 18],
                    [19, 20],
                    [21, 22]
                ]
            ],
            [
                [
                    [25, 26],
                    [27, 28],
                    [29, 30]
                ],
                [
                    [33, 34],
                    [35, 36],
                    [37, 38]
                ],
                [
                    [41, 42],
                    [43, 44],
                    [45, 46]
                ]
            ],
            [
                [
                    [49, 50],
                    [51, 52],
                    [53, 54]
                ],
                [
                    [57, 58],
                    [59, 60],
                    [61, 62]
                ],
                [
                    [65, 66],
                    [67, 68],
                    [69, 70]
                ]
            ]
        ]
    )
}

# Step 1.1 - Extract real and imaginary parts for Alice and Eve (assuming channels 0 and 1)
alice_real = new_data['cirs'][:, 0, :, 0]
alice_imag = new_data['cirs'][:, 0, :, 1]
eve_real = new_data['cirs'][:, 1, :, 0]
eve_imag = new_data['cirs'][:, 1, :, 1]

# Step 1.2 - Compute magnitudes
alice_mag = np.sqrt(alice_real**2 + alice_imag**2)
eve_mag = np.sqrt(eve_real**2 + eve_imag**2)

# Step 1.3 - Flatten the arrays
# Flatten features for Alice
alice_real_flat = alice_real.flatten()
alice_imag_flat = alice_imag.flatten()
alice_mag_flat = alice_mag.flatten()

# We flatten the arrays to make them suitable for the MinMaxScaler
# We flat all three(real, imaginary and magnitude) parts of the signal

# Step 1.4 - Combine features (making a feature vector)
alice_features = np.column_stack((alice_real_flat, alice_imag_flat, alice_mag_flat))

# Step 1.5 - Normalize the data (to train the model)
scaler = MinMaxScaler()
alice_features_scaled = scaler.fit_transform(alice_features)

# Step 2.1 - Train OCC-SVM for Alice
occ_svm = OneClassSVM(kernel='linear', nu=0.01)
occ_svm.fit(alice_features_scaled)
joblib.dump(occ_svm, 'occ_svm_model2.pkl')
joblib.dump(scaler, 'scaler2.pkl')

['scaler2.pkl']

In [4]:


# Example legitimate and illegitimate test data
legitimate_test_data = {
    'cirs': np.array(
        [
            [
                [
                    [1.1, 2.1],
                    [3.1, 4.1],
                    [5.1, 6.1]
                ],
                [
                    [9.1, 10.1],
                    [11.1, 12.1],
                    [13.1, 14.1]
                ],
                [
                    [17.1, 18.1],
                    [19.1, 20.1],
                    [21.1, 22.1]
                ]
            ]
        ]
    )
}

illegitimate_test_data = {
    'cirs': np.array(
        [
            [
                [
                    [50, 60],
                    [70, 80],
                    [90, 100]
                ],
                [
                    [110, 120],
                    [130, 140],
                    [150, 160]
                ],
                [
                    [170, 180],
                    [190, 200],
                    [210, 220]
                ]
            ]
        ]
    )
}

In [5]:
# Load the model and scaler
occ_svm = joblib.load('occ_svm_model2.pkl')
scaler = joblib.load('scaler2.pkl')

# Extract features for legitimate test data
legit_real = legitimate_test_data['cirs'][:, 0, :, 0]
legit_imag = legitimate_test_data['cirs'][:, 0, :, 1]
legit_mag = np.sqrt(legit_real**2 + legit_imag**2)
legit_features = np.column_stack((legit_real.flatten(), legit_imag.flatten(), legit_mag.flatten()))
legit_features_scaled = scaler.transform(legit_features)

# Extract features for illegitimate test data
illegit_real = illegitimate_test_data['cirs'][:, 0, :, 0]
illegit_imag = illegitimate_test_data['cirs'][:, 0, :, 1]
illegit_mag = np.sqrt(illegit_real**2 + illegit_imag**2)
illegit_features = np.column_stack((illegit_real.flatten(), illegit_imag.flatten(), illegit_mag.flatten()))
illegit_features_scaled = scaler.transform(illegit_features)

# Predict using OCC-SVM
legit_predictions = occ_svm.predict(legit_features_scaled)
illegit_predictions = occ_svm.predict(illegit_features_scaled)

# Check predictions (1 for inliers/legitimate, -1 for outliers/illegitimate)
print("Legitimate test data predictions:", legit_predictions)
print("Illegitimate test data predictions:", illegit_predictions)

legitimate_legit = legit_predictions == 1
illegitimate_legit = legit_predictions == -1
legitimate_illegit = illegit_predictions == 1
illegitimate_illegit = illegit_predictions == -1

print("Legitimate Test Data - Legitimate signals:", legitimate_legit.sum())
print("Legitimate Test Data - Illegitimate signals:", illegitimate_legit.sum())
print("Illegitimate Test Data - Legitimate signals:", legitimate_illegit.sum())
print("Illegitimate Test Data - Illegitimate signals:", illegitimate_illegit.sum())

Legitimate test data predictions: [-1 -1 -1]
Illegitimate test data predictions: [-1 -1 -1]
Legitimate Test Data - Legitimate signals: 0
Legitimate Test Data - Illegitimate signals: 3
Illegitimate Test Data - Legitimate signals: 0
Illegitimate Test Data - Illegitimate signals: 3
